In [ ]:
import sys

sys.path.append('../GSL')

In [ ]:
import os
from glob import glob
import yaml
from easydict import EasyDict as edict
import pickle

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import StandardScaler
import random

from torch_geometric.utils import erdos_renyi_graph, dense_to_sparse, to_dense_adj
from torch_geometric_temporal.signal import temporal_signal_split

from dataset.make_dataset_ecl import ECLDatasetLoader
from dataset.make_dataset_ett import ETTDatasetLoader # ETT -> 시간 정보
from dataset.make_dataset_exchange import ExchangeDatasetLoader
from dataset.make_dataset_wth import WTHDatasetLoader
from dataset.make_dataset_covid19 import COVID19DatasetLoader
from dataset.make_dataset_traffic import TrafficDatasetLoader

from dataset.utils import download_file, Info

from utils.scalers import Scaler
import gdown
from dataclasses import dataclass

In [ ]:
covid = pd.read_csv('../GSL/data/COVID19/time_series_covid19_confirmed_global.csv')

In [ ]:
covid

In [ ]:
country = ['US','Canada','Mexico','Russia',
           'United Kingdom','Italy','Germany','France',
           'Belarus','Brazil','Peru','Ecuador',
           'Chile','India','Turkey','Saudi Arabia',
           'Pakistan','Iran','Singapore','Qatar',
           'Bangladesh','United Arab Emirates','China','Japan','Korea, South']


In [ ]:
my_covid = covid[covid['Country/Region'].isin(country)].copy()

In [ ]:
my_covid.drop('Province/State', axis=1, inplace = True)

In [ ]:
my_covid_confimed = my_covid[my_covid.columns.tolist()[:1]+my_covid.columns.tolist()[3:]].copy()
my_covid_confimed.reset_index(drop=True, inplace=True)

In [ ]:
temp = pd.to_datetime(my_covid.columns[3:])

In [ ]:
my_covid_confimed.columns.values[1:] = temp

In [ ]:
df = my_covid_confimed.groupby('Country/Region').sum().copy()

In [ ]:
total_confirmed = df.sum(axis=1)

In [ ]:
country_lat_long_dict = {}

for cnt in country:
    lat = my_covid[covid['Country/Region']==cnt][['Lat', 'Long']].values
#     long = my_covid[covid['Country/Region']==cnt]['Long'].values

    country_lat_long_dict[cnt] = lat

In [ ]:
multi_region=[]

for cnt in country_lat_long_dict.keys():
    if len(country_lat_long_dict[cnt]) != 1:
        multi_region.append(cnt)

for key in multi_region:
    country_lat_long_dict.pop(key)

In [ ]:
country_lat_long_dict['Canada'] = np.array([[45.424721, -75.695000]])
country_lat_long_dict['United Kingdom'] = np.array([[51.509865, -0.118092]])
country_lat_long_dict['France'] = np.array([[48.864716, 2.349014]])
country_lat_long_dict['China'] = np.array([[39.916668, 116.383331]])

In [ ]:
for cnt in country_lat_long_dict.keys():
    country_lat_long_dict[cnt] = [float(country_lat_long_dict[cnt][0][0]), float(country_lat_long_dict[cnt][0][1])]

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[20,0], tiles="OpenStreetMap", zoom_start=2)

for cnt in country_lat_long_dict:
    folium.Circle(
        location=[country_lat_long_dict[cnt][0], country_lat_long_dict[cnt][1]],
        popup=cnt,
        radius = float(total_confirmed[cnt])/20000,
        color = 'crimson',
        fill = True,
        fill_color = 'crimson').add_to(m)
    
#     folium.Marker(location=[country_lat_long_dict[cnt][0], country_lat_long_dict[cnt][1]],
#                 popup=cnt,
#                 icon=folium.DivIcon(html=f"""<div style="font-family: courier new; color: crimson">{cnt}</div>""")
#                 ).add_to(m)
    

# folium.PolyLine(locations=list(country_lat_long_dict.values()),
#                tooltip='Polyline',
#                color = 'crimson',
#                weight=1,
#                dash_array='10').add_to(m)

m.save('./map.html')
m